## тестирование стратегии на промежутке дней (год)

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import os
from dotenv import load_dotenv
from tinkoff.invest import Client

from tinkoff.invest import CandleInterval
from datetime import datetime

load_dotenv()

TOKEN = os.getenv("INVEST_TOKEN")

TICKER = 'RNFT'
FIGI = 'BBG00F9XX7H4'

In [77]:
from trader_bot import ScalpingBot
from historical_data import HistoricalDataHandler

from test_env.client_test_env import ClientTestEnvHelper
from test_env.logger_test_env import LoggerTestEnvHelper
from test_env.time_test_env import TimeTestEnvHelper
from test_env.accounting_test_env import AccountingTestEnvHelper

def test_algorythm(
        last_test_date = '2024-03-15',
        test_days_num = 251,
        profit_steps=5, 
        candles_count=4, 
        sleep_trading=5 * 60, 
        sleep_no_trade=300, 
        no_operation_timeout_seconds=300,
        ):
    time_helper = TimeTestEnvHelper()
    
    logger_helper = LoggerTestEnvHelper(time_helper, do_printing=False)
    
    client_helper = ClientTestEnvHelper(TICKER, logger_helper, time_helper)
    client_helper.set_ticker_params(1, 0.1, FIGI, 'RUR')
    
    accounting_helper = AccountingTestEnvHelper(client_helper)
    
    data_handler = HistoricalDataHandler(TOKEN, FIGI, TICKER)
    
    days_list = data_handler.get_days_list(last_test_date, test_days_num)
    
    # создаем бота с настройками
    bot = ScalpingBot(
        TOKEN, 
        TICKER,
    
        profit_steps=profit_steps,
        candles_count=candles_count,
        
        sleep_trading=sleep_trading,
        sleep_no_trade=sleep_no_trade,
        no_operation_timeout_seconds=no_operation_timeout_seconds,
    
        time_helper=time_helper,
        logger_helper=logger_helper,
        client_helper=client_helper,
        accounting_helper=accounting_helper,
    )
    
    balance = 0
    success_days = 0
    balance_change_list = []
    
    # закручиваем цикл по датам
    for test_date in days_list:
    
        # прогоняем по дню (-3 часа для компенсации часового сдвига)
        date_from = datetime.strptime(test_date + ' 07:30', "%Y-%m-%d %H:%M")   # 10:30
        date_to   = datetime.strptime( test_date + ' 15:30', "%Y-%m-%d %H:%M")  # 18:30
        
        # задаем параметры дня
        time_helper.set_current_time(date_from)
        bot.reset_last_operation_time()
        client_helper.set_candles_list(data_handler.get_candles(test_date))
        
        accounting_helper.reset()
    
        # Использование итератора для вывода каждой пары час-минута
        for dt in data_handler.get_hour_minute_pairs(date_from, date_to):
            # задаем время
            time_helper.set_time(dt)
            
            candle = client_helper.get_candle(dt)
            if candle is None:
                logger_helper.error(f"No candle for {dt}")        
                continue
            
            # задаем текущее значение свечи
            client_helper.set_current_candle(candle)
            
            # анализируем заявки - успешные помечаем
            if bot.buy_order:
                buy_price = client_helper.quotation_to_float(bot.buy_order.initial_order_price)
                low_buy_price = client_helper.quotation_to_float(candle.low)
                if buy_price >= low_buy_price:
                    client_helper.buy_order_executed = True
                    client_helper.buy_order_executed_on_border = buy_price == low_buy_price
        
            if bot.sell_order:
                sell_price = client_helper.quotation_to_float(bot.sell_order.initial_order_price)
                high_sell_price = client_helper.quotation_to_float(candle.high)
                if sell_price <= high_sell_price:
                    client_helper.sell_order_executed = True
                    client_helper.sell_order_executed_on_border = sell_price == high_sell_price
        
            # если пора просыпаться
            if time_helper.is_time_to_awake():
                # print(dt.strftime("%H:%M"))
                # запускаем итерацию торгового алгоритма
                bot.run_iteration()
        
        bot.stop()
        
        balance_change = round(accounting_helper.sum, 2)
        balance = round(balance + balance_change, 2)
        
        if balance_change > 0:
            success_days += 1
            
        balance_change_list.append(balance_change)
        
        # выводим итоговый результат
        # orders_border_cnt = client_helper.total_completed_orders_on_border
        # orders_cnt = client_helper.total_completed_orders
        # print(f"{test_date}: {balance} // {balance_change} {client_helper.currency}, "
        #       f"операций: {orders_cnt}, "
        #       f"граничных {orders_border_cnt} "
        #       f"({round(100 * orders_border_cnt / orders_cnt, 2) if orders_cnt else 0}%)")
        
    return {
        'balance': balance,
        'balance_change_avg': round(sum(balance_change_list) / test_days_num, 2),
        'success_days': success_days,
        'success_p': round(success_days / test_days_num, 2),
        'profit_steps': profit_steps,
        'candles_count': candles_count,
        'sleep_trading': sleep_trading,
        'no_op_seconds': no_operation_timeout_seconds,
    }
        

# print(test_algorythm(
#     last_test_date = '2024-03-15',
#     test_days_num = 5, # 251,
# 
#     profit_steps=5, 
#     candles_count=4, 
# 
#     sleep_trading=5 * 60, 
#     sleep_no_trade=300, 
#     no_operation_timeout_seconds=300,
# ))

results = []

# for profit_steps in [2]:
#     for candles_count in [2]:
#         for sleep_trading in [1*60]:
#             for no_operation_timeout_seconds in [1*60]:
for profit_steps in [2, 3, 4, 5, 6, 7]:
    for candles_count in [2, 3, 4, 5, 6]:
        for sleep_trading in [1*60, 2*60, 3*60, 4*60, 5*60, 6*60, 7*60, 9*60, 10*60]:
            for no_operation_timeout_seconds in [1*60, 2*60, 3*60, 4*60, 5*60, 10*60, 15*60, 20*60, 30*60]:
                res = test_algorythm(
                    last_test_date = '2024-03-15',
                    test_days_num = 65, # 251,
                
                    profit_steps=profit_steps, 
                    candles_count=candles_count, 
                
                    sleep_trading=sleep_trading, 
                    sleep_no_trade=300, 
                    no_operation_timeout_seconds=300,
                )
                print('.', end='')
                results.append(res)
            
sorted_results = sorted(results, key=lambda x: x['balance'], reverse=True)

print()
for item in sorted_results:
    print(item)


.
{'balance': -490.3, 'balance_change_avg': -5.45, 'success_days': 2, 'success_p': 0.02, 'profit_steps': 2, 'candles_count': 2, 'sleep_trading': 60, 'no_op_seconds': 300}


In [73]:
sorted_results

[{'balance': -69.8,
  'balance_change_avg': -2.33,
  'success_days': 2,
  'success_p': 0.07,
  'profit_steps': 3,
  'candles_count': 3,
  'sleep_trading': 60,
  'no_op_seconds': 300},
 {'balance': -73.7,
  'balance_change_avg': -2.46,
  'success_days': 3,
  'success_p': 0.1,
  'profit_steps': 3,
  'candles_count': 2,
  'sleep_trading': 60,
  'no_op_seconds': 300},
 {'balance': -145.1,
  'balance_change_avg': -4.84,
  'success_days': 2,
  'success_p': 0.07,
  'profit_steps': 2,
  'candles_count': 3,
  'sleep_trading': 60,
  'no_op_seconds': 300},
 {'balance': -156.9,
  'balance_change_avg': -5.23,
  'success_days': 1,
  'success_p': 0.03,
  'profit_steps': 2,
  'candles_count': 2,
  'sleep_trading': 60,
  'no_op_seconds': 300}]